## Step 1: The preliminaries
### 1(a) Import the libraries 

In [ ]:
import os
import numpy as np
import scipy
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
import statsmodels.formula.api as smf
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns

### 1(b) Load all the data

In [ ]:
%run 'load_data_common.py'

# First we are going to explore the datasets for 4 groups:  
## (A) Control  
## (B) Bay K8644  
## (C) Verapamil
## (D) Dofetilide

In [ ]:
Control = df_features.loc[(df_features['drug'] == 'Control')]
BayK = df_features.loc[(df_features['drug'] == 'Bay_K')]
Verapamil = df_features.loc[(df_features['drug'] == 'Verapamil')] #& (df_features['dose'] != '10uM') & (df_features['dose'] != '100uM')  ]
Dofetilide = df_features.loc[(df_features['drug'] == 'Dofetilide')]
df_reduced = pd.concat([Control,BayK,Verapamil,Dofetilide])

# Let's take a look at their (all doses) distributions first - for a couple of APD metrics

## $APD_{80}$

In [ ]:
sns.catplot(x='drug', y='voltage_apd80', data=df_reduced[['drug','voltage_apd80']], dodge=True, kind='violin', aspect=3)

In [ ]:
sns.barplot(x = "drug", y = "voltage_apd80", data = df_reduced[['drug',"voltage_apd80"]], estimator = "mean", errorbar = 'se')

## $APD_{50}$

In [ ]:
sns.catplot(x='drug', y='voltage_apd50', data=df_reduced[['drug','voltage_apd50']], dodge=True, kind='violin', aspect=3)

## $APD_{30}$

In [ ]:
sns.catplot(x='drug', y='voltage_apd30', data=df_reduced[['drug','voltage_apd30']], dodge=True, kind='violin', aspect=3)

# What can we say about these distributions?  
## What explanations might you offer?

In [ ]:
Control.describe()

In [ ]:
BayK.describe()

In [ ]:
Verapamil.describe()

In [ ]:
Dofetilide.describe()

# Is the variability in APD$_{90}$ for Bay K8644 and Verapamil due to dose?

## Let's look at the Control distribution as a reference first

In [ ]:
Control['voltage_apd80'].plot(kind='kde', xlim=[0,3000], ylim=[0,0.02])
plt.legend(['$APD_{90}$'], title='Control ')

## Bay K8644

In [ ]:
BayK.groupby('dose')['voltage_apd80'].plot(kind='kde', xlim=[0,3000], ylim=[0,0.02])
dose = ['1000nM','100nM','10nM','1nM','baseline']
plt.legend(dose, title='Bay K8644 dose')

## Verapamil

In [ ]:
Verapamil.groupby('dose')['voltage_apd80'].plot(kind='kde', xlim=[0,3000], ylim=[0,0.04])
dose = ['0.1uM','100uM','10uM','1uM','baseline']
plt.legend(dose, title='Verapamil dose')

## Dofetilide

In [ ]:
Dofetilide.groupby('dose')['voltage_apd80'].plot(kind='kde', xlim=[0,3000], ylim=[0,0.03])
dose = ['0.1nM','100nM','10nM','1nM','baseline']
plt.legend(dose, title='Dofetilide dose')

# Could some of these data qualify as statistical outliers?

## Some tests to define outliers based on Gaussian assumptions

## First let's take a look at the data without smoothing

In [ ]:
Verapamil.groupby('dose')['voltage_apd80'].plot(kind='hist')
dose = ['0.1uM','100uM','10uM','1uM','baseline']
plt.legend(dose, title='Verapamil dose')

### First let's set our $\alpha$ threshold for all the tests:

In [ ]:
alpha = 0.05

### (1) Grubbs test (single outlier suspected) - can be used iteratively, but not best practice

#### Now see if any single data point qualifies in the 10 uM set 

In [ ]:
Verapamil_10um = Verapamil.loc[(Verapamil["dose"] == "10uM")]
Verapamil_10_for_outliers = Verapamil_10um["voltage_apd80"].to_numpy()
Verapamil_10_for_outliers = np.delete(Verapamil_10_for_outliers, [2,3,6]) # editing for parsing, remove if fixed
print("\nAll Verapamil data at 10 uM:\n",Verapamil_10_for_outliers)
Grubbs_corrected = sp.outliers_grubbs(Verapamil_10_for_outliers, alpha = alpha)
print("\nData after correction for the Grubbs criterion are:\n",Grubbs_corrected)
set_diff = np.setdiff1d(Verapamil_10_for_outliers, Grubbs_corrected)
print("\nThe excluded data are:\n", set_diff)

In [ ]:
Verapamil_100um = Verapamil.loc[(Verapamil["dose"] == "100uM")]
Verapamil_100_for_outliers = Verapamil_100um["voltage_apd80"].to_numpy()
Verapamil_100_for_outliers = np.delete(Verapamil_100_for_outliers, [7]) # editing for parsing, remove if fixed
Verapamil_100_for_outliers = Verapamil_100_for_outliers[~np.isnan(Verapamil_100_for_outliers)]
print("\nAll Verapamil data at 100 uM:\n",Verapamil_100_for_outliers)
Grubbs_corrected = sp.outliers_grubbs(Verapamil_100_for_outliers, alpha = alpha)
print("\nData after correction for the Grubbs criterion are:\n",Grubbs_corrected)
set_diff = np.setdiff1d(Verapamil_100_for_outliers, Grubbs_corrected)
print("\nThe excluded data are:\n", set_diff)

### (2) Tietjen-Moore test (specified number of outliers suspected)

#### See if it is possible to correct these datasets by choosing the right number of specified outliers (num_outliers).  
#### Can you start by trying to identify the number in the histogram above? 
#### First for the 10 uM data

In [ ]:
num_outliers = 2

print("\nAll Verapamil data at 10 uM:\n",Verapamil_10_for_outliers)
TM_10_corrected = sp.outliers_tietjen(Verapamil_10_for_outliers, num_outliers, alpha = alpha)
print("\nData after correction for the Tietjen-Moore criterion at n = (",num_outliers,") are:\n",TM_10_corrected)
set_diff = np.setdiff1d(Verapamil_10_for_outliers, TM_10_corrected)
print("\nThe excluded data are:\n", set_diff)

#### Now for 100 uM

In [ ]:
num_outliers = 3

print("\nAll Verapamil data at 100 uM:\n",Verapamil_100_for_outliers)
TM_100_corrected = sp.outliers_tietjen(Verapamil_100_for_outliers, num_outliers, alpha = alpha) 
print("\nData after correction for the Tietjen-Moore criterion at n = (",num_outliers,") are:\n",TM_100_corrected)
set_diff = np.setdiff1d(Verapamil_100_for_outliers, TM_100_corrected)
print("\nThe excluded data are:\n", set_diff)

### (3) Extreme studentized deviate test (specified maximum number of outliers)  
#### This test only assumes a specified maximum number of outliers

In [ ]:
max_outliers = 0

print("\nAll Verapamil data at 10 uM:\n",Verapamil_10_for_outliers,"\n")
GESD_10_corrected = sp.outliers_gesd(Verapamil_10_for_outliers,max_outliers)
print(sp.outliers_gesd(Verapamil_10_for_outliers, outliers = max_outliers, alpha = alpha, report = True))
print("\nData after correction for up to n = (",max_outliers,") are:\n",GESD_10_corrected)
set_diff = np.setdiff1d(Verapamil_10_for_outliers, GESD_10_corrected)
print("\nThe excluded data are:\n", set_diff)

In [ ]:
max_outliers = 0

print("\nAll Verapamil data at 100 uM:\n",Verapamil_100_for_outliers,"\n")
GESD_100_corrected = sp.outliers_gesd(Verapamil_100_for_outliers,max_outliers)
print(sp.outliers_gesd(Verapamil_100_for_outliers, outliers = max_outliers, alpha = alpha, report = True))
print("\nData after correction for up to n = (",max_outliers,") are:\n",GESD_100_corrected)
set_diff = np.setdiff1d(Verapamil_100_for_outliers, GESD_100_corrected)
print("\nThe excluded data are:\n", set_diff)

## (4) What about classic z-score thresholding?  
### How many standard deviations must you account for to remove the outlying data?

In [ ]:
z_thresh = 1.0

print("\nAll Verapamil data at 10 uM:\n",Verapamil_10_for_outliers)
z = np.abs(stats.zscore(Verapamil_10_for_outliers))
print("\nAll Verapamil z-scores 10 uM:\n",z)
z_10_corrected = Verapamil_10_for_outliers[(z<z_thresh)]
print("\nData after z >", z_thresh, "correction:\n", z_10_corrected)
set_diff = np.setdiff1d(Verapamil_10_for_outliers, z_10_corrected)
print("\nThe excluded data are:\n", set_diff)

### What about non-parametric options?

### (4) Interquartile range discriminator

In [ ]:
print("\nAll Verapamil data at 10 uM:\n",Verapamil_10_for_outliers)
threshold = 1.5
IQR_10_corrected = sp.outliers_iqr(Verapamil_10_for_outliers, ret = 'filtered', coef = threshold)
print("\nData after IQR correction:\n",IQR_10_corrected)
IQR_10_excluded = sp.outliers_iqr(Verapamil_10_for_outliers, ret = 'outliers', coef = threshold)
print("\nThe excluded data are:\n",IQR_10_excluded)

In [ ]:
print("\nAll Verapamil data at 100 uM:\n",Verapamil_100_for_outliers)
threshold = 1.5
IQR_100_corrected = sp.outliers_iqr(Verapamil_100_for_outliers, ret = 'filtered', coef = threshold)
print("\nData after IQR correction:\n",IQR_100_corrected)
IQR_100_excluded = sp.outliers_iqr(Verapamil_100_for_outliers, ret = 'outliers', coef = threshold)
print("\nThe excluded data are:\n",IQR_100_excluded)

## What do you conclude about these unbiased methods for data exclusion in the small-sample setting? 